# Loading in a user

In [3]:
import bandicoot as bc
user = bc.read_csv('user', 'data/', 'data/towers.csv')

[x] 21 records from 2014-07-01 11:11:27 to 2014-07-20 11:26:27
[x] 5 contacts
[ ] No attributes stored
[x] 17 antennas
[x] Has home
[x] Has texts
[x] Has calls


In [4]:
import pandas
from collections import Counter

In [21]:
antennas = [(r.position.antenna,) + r.position._get_location(user) for r in user.records]
df_antennas = pandas.DataFrame([a + (count, ) for a, count in Counter(antennas).items()])
df_antennas.columns = ['id', 'latitude', 'longitude', 'interactions']
df_antennas.to_csv('viz/mobility_view/antenna.csv', index=False)

antennas_id = [a[0] for a in antennas]
transitions = Counter(zip(antennas_id, antennas_id[1:]))
df_transitions = pandas.DataFrame([a + (count, ) for a, count in transitions.items()])
df_transitions.columns = ['source', 'target', 'amount']
df_transitions.to_csv('viz/mobility_view/transitions.csv', index=False)

In [22]:
%%html
<iframe src="viz/mobility_view/index.html" width="100%" height="400px">

## Event Timeline

In [16]:
timeline = pandas.DataFrame([(r.datetime, ('inc_' if r.direction == 'in' else 'out_') + r.interaction, r.call_duration) for r in user.records])
timeline.columns = ['time', 'type', 'call_duration']
timeline['time'] = timeline['time'].map(lambda x: x.strftime('%d-%m-%y %H:%M'))
timeline.to_csv('viz/event_timeline/timeseries.csv', index=False)

In [17]:
%%html
<iframe src="viz/event_timeline/index.html" width="100%" height="400px">

## Network view

In [27]:
# Nodes
pandas.DataFrame([0] + [0 + (i is None) for (key, i) in user.network.items() if key != user.name], columns=['no_network_info']).to_csv('viz/network_view/nodes.csv', index=False)

In [52]:
m_texts = bc.network._interaction_matrix_text(user)
m_calls = bc.network._interaction_matrix_call(user)
nb_users = len(user.network)

df_links = pandas.DataFrame([(i, j, m_texts[j][i], m_texts[i][j], m_calls[j][i], m_calls[i][j]) for i in range(nb_users) for j in range(nb_users)]).fillna(0)
df_links.columns = ['source', 'target', 'inc_calls', 'out_calls', 'inc_texts', 'out_texts']
df_links = df_links[df_links[['inc_calls', 'out_calls', 'inc_texts', 'out_texts']].sum(axis=1) > 0]
df_links = df_links[df_links.source < df_links.target]
df_links.to_csv('viz/network_view/links.csv', index=False)

In [70]:
nodes = set(df_links.source) | set(df_links.target)
pandas.DataFrame([int(n in df_links.source) for n in nodes], columns=['no_network_info']).to_csv('viz/network_view/nodes.csv', index=False)

In [71]:
%%html
<iframe src="viz/network_view/index.html" width="100%" height="400px">